In [25]:

# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'main_models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models



In [54]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    print("stacked_data")
    print(inputX.shape)
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    print(stackX.shape)
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    print(stackX)
    return stackX


In [55]:


# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    print("fit_stacked_model")
    print(inputX.shape)
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = svm.SVC(kernel='linear')
    model.fit(stackedX, inputy)
    return model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    print("stacked_pred")
    print(inputX.shape)
    stackedX = stacked_dataset(members, inputX)
    print("stackedX",stackedX.shape)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [56]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
#import tensorflow as tf
dataset = pd.read_csv('updated_test.csv')
dataset = dataset[ dataset['YEAR']>1980 ]
dataset = dataset.dropna()
X = dataset.iloc[:,[0,3,4,6,7]].values
y = dataset.iloc[:,5].values
print(X.shape)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
print(X.shape)
onehotencoder = OneHotEncoder(categorical_features = [0,1,3])
X = onehotencoder.fit_transform(X).toarray()
print(X.shape)

X = X[:,1:]
print(X.shape)
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0)
onehotencoder = OneHotEncoder(categorical_features = [0])
y_train = np.reshape(y_train,(-1,1))
y_train = onehotencoder.fit_transform(y_train).toarray()
print(X.shape)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print(X.shape)
print("x train",X_train.shape)
print("x test",X_test.shape)

(4187, 5)
(4187, 5)
(4187, 48)
(4187, 47)
(4187, 47)
(4187, 47)
x train (3349, 47)
x test (838, 47)


In [57]:
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
print("y test",y_test.shape)
print("x test",X_test.shape)
for model in members:
    testy_enc = to_categorical(y_test)
    print("y test enc",testy_enc.shape)
    _, acc = model.evaluate(X_test, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
# fit stacked model using the ensemble
model = fit_stacked_model(members, X_test, y_test)
print(X_test.shape)
# evaluate model on test set
yhat = stacked_prediction(members, model, X_test)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

>loaded main_models/model_1.h5
>loaded main_models/model_2.h5
>loaded main_models/model_3.h5
>loaded main_models/model_4.h5
>loaded main_models/model_5.h5
Loaded 5 models
y test (838,)
x test (838, 47)
y test enc (838, 6)
Model Accuracy: 0.947
y test enc (838, 6)
Model Accuracy: 0.942
y test enc (838, 6)
Model Accuracy: 0.947
y test enc (838, 6)
Model Accuracy: 0.945
y test enc (838, 6)
Model Accuracy: 0.945
fit_stacked_model
(838, 47)
stacked_data
(838, 47)
(838, 6, 5)
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [3.3324632e-01 2.0566601e-01 2.5782811e-02 ... 2.0181219e-35
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 ... 0.00000

KeyError: 'jan'

In [1]:
    month='06'
    state="TN"
    if(month == '01' or month == '02'):
        month = 'Jan-Feb'
        duration = 2
    elif(month == '03' or month == '04' or month == '05'):
        month = 'Mar-May'
        duration = 3
    elif(month == '06' or month == '07' or month == '08' or month == '09'):
        month = 'Jun-Sep'
        duration = 4
    elif(month == '10' or month == '11' or month == '12'):
        month = 'Oct-Dec'
        duration = 3

    #for the state code
    if(state == 'WB'):
        state = 'GANGETIC WEST BENGAL'
    elif(state == 'AN'):
        state = 'ANDAMAN & NICOBAR ISLANDS'
    elif(state == 'AR'):
        state = 'ARUNACHAL PRADESH'
    elif(state == 'AS'):
        state = 'ASSAM & MEGHALAYA'
    elif(state == 'BR'):
        state = 'BIHAR'
    elif(state == 'CG'):
        state = 'CHATTISGARH'
    elif(state == 'AD'):
        state = 'COASTAL ANDHRA PRADESH'
    elif(state == 'KA'):
        state = 'COASTAL KARNATAKA'
    elif(state == 'MP'):
        state = 'EAST MADHYA PRADESH'
    elif(state == 'RJ'):
        state = 'EAST RAJASTHAN'
    elif(state == 'UP'):
        state = 'EAST UTTAR PRADESH'
    elif(state == 'GJ'):
        state = 'GUJARAT REGION'
    elif(state == 'DL'):
        state = 'HARYANA DELHI & CHANDIGARH'
    elif(state == 'HP'):
        state = 'HIMACHAL PRADESH'
    elif(state == 'JK'):
        state = 'JAMMU & KASHMIR'
    elif(state == 'JH'):
        state = 'JHARKHAND'
    elif(state == 'KL'):
        state = 'KERALA'
    elif(state == 'GA'):
        state = 'KONKAN & GOA'
    elif(state == 'LD'):
        state = 'LAKSHWADEEP'
    elif(state == 'MH'):
        state = 'MADHYA MAHARASHTRA'
    elif(state == 'MT'):
        state = 'MATATHWADA'
    elif(state == 'MN'):
        state = 'NAGA MANI MIZO TRIPURA'
    elif(state == 'KA'):
        state = 'NORTH INTERIOR KARNATAKA'
    elif(state == 'OD'):
        state = 'ORISSA'
    elif(state == 'PB'):
        state = 'PUNJAB'
    elif(state == 'RS'):
        state = 'RAYALSEEMA'
    elif(state == 'SK'):
        state = 'SAURASHTRA & KUTCH'
    elif(state == 'KA'):
        state = 'SOUTH INTERIOR KARNATAKA'
    elif(state == 'SK'):
        state = 'SUB HIMALAYAN WEST BENGAL & SIKKIM'
    elif(state == 'TN'):
        state = 'TAMIL NADU'
    elif(state == 'TS'):
        state = 'TELANGANA'
    elif(state == 'UK'):
        state = 'UTTARAKHAND'
    elif(state == 'VD'):
        state = 'VIDARBHA'
    elif(state == 'MP'):
        state = 'WEST MADHYA PRADESH'
    elif(state == 'RJ'):
        state = 'WEST RAJASTHAN'
    elif(state == 'UP'):
        state = 'WEST UTTAR PRADESH'

    #for the terrain
    if(state == 'GANGETIC WEST BENGAL'):
        terrain = 'Coastal-plateau'
    elif(state == 'ANDAMAN & NICOBAR ISLANDS'):
        terrain = 'Island'
    elif(state == 'ARUNACHAL PRADESH'):
        terrain = 'Hilly'
    elif(state == 'ASSAM & MEGHALAYA'):
        terrain = 'Hilly'
    elif(state == 'BIHAR'):
        terrain = 'Plain-land'
    elif(state == 'CHATTISGARH'):
        terrain = 'Hilly'
    elif(state == 'COASTAL ANDHRA PRADESH'):
        terrain = 'Coastal'
    elif(state == 'COASTAL KARNATAKA'):
        terrain = 'Coastal'
    elif(state == 'EAST MADHYA PRADESH'):
        terrain = 'Everything'
    elif(state == 'EAST RAJASTHAN'):
        terrain = 'Desert'
    elif(state == 'EAST UTTAR PRADESH'):
        terrain = 'Rugged'
    elif(state == 'GUJARAT REGION'):
        terrain = 'Desert/marsh'
    elif(state == 'HARYANA DELHI & CHANDIGARH'):
        terrain = 'Plain-land'
    elif(state == 'HIMACHAL PRADESH'):
        terrain = 'Hilly'
    elif(state == 'JAMMU & KASHMIR'):
        terrain = 'Hilly'
    elif(state == 'JHARKHAND'):
        terrain = 'Forest'
    elif(state == 'KERALA'):
        terrain = 'Coastal'
    elif(state == 'KONKAN & GOA'):
        terrain = 'Hilly/coastal'
    elif(state == 'LAKSHWADEEP'):
        terrain = 'Island'
    elif(state == 'MADHYA MAHARASHTRA'):
        terrain = 'Plain-land'
    elif(state == 'MATATHWADA'):
        terrain = 'Barren'
    elif(state == 'NAGA MANI MIZO TRIPURA'):
        terrain = 'Hilly'
    elif(state == 'NORTH INTERIOR KARNATAKA'):
        terrain = 'Coastal'
    elif(state == 'ORISSA'):
        terrain = 'Coastal'
    elif(state == 'PUNJAB'):
        terrain = 'Plain-land'
    elif(state == 'RAYALSEEMA'):
        terrain = 'Plain-land'
    elif(state == 'SAURASHTRA & KUTCH'):
        terrain = 'Hilly'
    elif(state == 'SOUTH INTERIOR KARNATAKA'):
        terrain = 'Coastal'
    elif(state == 'SUB HIMALAYAN WEST BENGAL & SIKKIM'):
        terrain = 'Hilly'
    elif(state == 'TAMIL NADU'):
        terrain = 'Hilly/coastal'
    elif(state == 'TELANGANA'):
        terrain = 'Hilly/plain'
    elif(state == 'UTTARAKHAND'):
        terrain = 'Hilly'
    elif(state == 'VIDARBHA'):
        terrain = 'Plain-land'
    elif(state == 'WEST MADHYA PRADESH'):
        terrain = 'Plain-land'
    elif(state == 'WEST RAJASTHAN'):
        terrain = 'Desert'
    elif(state == 'WEST UTTAR PRADESH'):
        terrain = 'Hilly'
    print(month,state,terrain)

Jun-Sep TAMIL NADU Hilly/coastal


In [3]:
import pandas as pd
import numpy as np
RainfallData = pd.read_csv('RainfallData.csv')
given_state = RainfallData['STATE']=='TN'
RainfallData = RainfallData[given_state]
prec = RainfallData[month].iloc[0]


In [4]:
x = np.array([state,month,prec,terrain,'humid']);
print(x)

['TAMIL NADU' 'Jun-Sep' '330.840625' 'Hilly/coastal' 'humid']


In [5]:
x=np.reshape(x,(-1,5))
print(x)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_x = LabelEncoder()


[['TAMIL NADU' 'Jun-Sep' '330.840625' 'Hilly/coastal' 'humid']]


In [6]:
x[:,0] = labelencoder_x.fit_transform(x[:, 0])
x[:, 1] = labelencoder_x.fit_transform(x[:, 1])
x[:, 3] = labelencoder_x.fit_transform(x[:, 3])
x[:, 4] = labelencoder_x.fit_transform(x[:, 4])
onehotencoder = OneHotEncoder(categorical_features = [0,1,3])
print(x.shape)
x= onehotencoder.fit_transform(x).toarray()
print(x.shape)
x = x[:,1:]

(1, 5)
(1, 5)


In [168]:
print(x.shape)

(1, 4)


In [7]:
from sklearn.preprocessing import StandardScaler
#normalization
#val-mean/n
sc_X = StandardScaler()
x = sc_X.fit_transform(x)
print(x.shape)

(1, 4)
